# Making tools to use covariate xarrays with a Keras model

### Loading covariates and targets

These should already exist in a bunch of pickle files

In [2]:
import pickle
with open('site_and_points.pkl','rb') as f:
    final_df = pickle.load(f)

In [19]:
with open("quantile_img.pkl","rb") as f:
    quant_raster = pickle.load(f)

In [168]:
#seasonal means
with open("mean_seasonal_FC.pkl","rb") as f:
    seasonal_means = pickle.load(f)

In [172]:
PV_seas = seasonal_means.sel(variable='PV').rename({'season':'channel'})

In [173]:
PV_seas

<xarray.DataArray (y: 886, x: 659, channel: 4)>
array([[[61.97203 , 56.99014 , 60.784644, 50.163485],
        [59.686388, 56.539571, 62.130111, 49.612824],
        ...,
        [52.804433, 45.763796, 51.412143, 43.88654 ],
        [56.514986, 48.027604, 53.945511, 45.846932]],

       [[59.696613, 57.998884, 62.081184, 50.653793],
        [58.84326 , 54.444835, 58.516161, 50.029311],
        ...,
        [54.473558, 47.502116, 53.673721, 44.763188],
        [57.525812, 51.180986, 58.85907 , 48.93493 ]],

       ...,

       [[74.482983, 52.336733, 62.816531, 67.265901],
        [76.309705, 53.302597, 62.732599, 68.337951],
        ...,
        [59.712529, 59.333012, 70.1329  , 53.606898],
        [55.674188, 54.775554, 69.946958, 49.202397]],

       [[75.849694, 51.32138 , 62.854182, 68.617417],
        [75.589411, 52.836265, 63.55757 , 65.914851],
        ...,
        [60.414193, 60.326271, 70.482568, 54.457972],
        [58.453094, 58.396196, 70.109053, 51.88663 ]]])
Coordinates:
  

In [165]:
quant_raster.channel

<xarray.DataArray 'channel' (channel: 7)>
array([0.01, 0.1 , 0.25, 0.5 , 0.75, 0.9 , 0.99])
Coordinates:
  * channel  (channel) float64 0.01 0.1 0.25 0.5 0.75 0.9 0.99

In [253]:
import xarray as xr
tpi = xr.open_rasterio('SOC_geotiff/TPI_ablers.tif')
saga = xr.open_rasterio('SOC_geotiff/sagawetness_albers.tif')
#need to make nodata non-numeric for standardisation to work correctly
tpi = tpi.where(tpi!=tpi.nodatavals[0])
saga = saga.where(saga!=saga.nodatavals)

In [7]:
final_df.head()

,SampleID,Easting,Northing,TC,Method,Year,points
0,2001_A1.2,338014.132,6370645.57,0.981252,CNS,2001,"Geometry({'type': 'Point', 'coordinates': (178..."
1,A1MIR,338014.132,6370645.57,0.600364,MIR,2001,"Geometry({'type': 'Point', 'coordinates': (178..."
2,2001_A6.2,338068.776,6370868.38,0.866419,CNS,2001,"Geometry({'type': 'Point', 'coordinates': (178..."
3,A6MIR,338068.776,6370868.38,1.187051,MIR,2001,"Geometry({'type': 'Point', 'coordinates': (178..."
4,2001_A11.2,338182.533,6370550.16,0.772519,CNS,2001,"Geometry({'type': 'Point', 'coordinates': (178..."


Now we have a DataFrame with all the position and target information about the site measurements, and raster maps of the TPI and SAGA wetness, along with quantiles of photosynthetic vegetation cover observed by Landsat. We should combine these separate rasters into one huge multi-channel raster, then write a function to select from this raster and produce a 'window' around a site measurement for input into the neural network.

In [254]:
topographic = xr.concat((saga,tpi),dim='band').rename({'band':'channel'})

In [255]:
topographic.shape

(2, 886, 659)

In [256]:
quant_raster = quant_raster.rename({'quantile':'channel'})

ValueError: cannot rename 'quantile' because it is not a variable or dimension in this dataset

In [257]:
covars = xr.concat((topographic,quant_raster,PV_seas),dim='channel')

In [258]:
# remove incompatible metadata in the dimension that was concatenated
import numpy as np; covars['channel'] = np.arange(len(covars['channel']))

In [259]:
covars.channel

<xarray.DataArray 'channel' (channel: 13)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
Coordinates:
  * channel  (channel) int64 0 1 2 3 4 5 6 7 8 9 10 11 12

### Function to create training/validation samples

This function takes a point (x,y) and a specified buffer around it (in pixels), then returns a trimmed raster of the covariates around the point. It should deal with cases where the buffer zone intersects the edge of the covariate raster map.

In [311]:
#determine resolution of rasters by differencing the spatial dimensions.
#ensure that if the raster contains covariates from different sources that they are coregistered
#to the same spatial coordinate sets otherwise this won't work and you'll end up with a bunch of
#NaNs in your underlying numpy arrays.
xres = covars.x[1]-covars.x[0]
yres = covars.y[1]-covars.y[0]

def sample_raster(row,bufferx=10,buffery=10):
    LL = row['points']
    sitex = LL.coords[0][0]
    sitey = LL.coords[0][1]
    
    x = np.arange(sitex-bufferx*xres,sitex+(bufferx+1)*xres,xres)
    y = np.arange(sitey-buffery*yres,sitey+(buffery+1)*yres,yres)
    
    sample_array = covars.reindex(x=x,method='nearest',tolerance=abs(xres/2)).reindex(y=y,method='nearest',tolerance=abs(yres/2))
    
    sample_array = sample_array.fillna(0)
    
    return sample_array.data
    
    
        

It may help to standardise the inputs for training. We can do this simply using built-in features of xarray before generating training samples. We can then either impute missing values (NaNs) on-the-fly using Keras or do it using our sample generating function while creating the training/validation set. It is less costly to do the latter because once it's done it will not need to be done again.

In [262]:
covars = (covars - covars.mean(dim=['x','y']))/covars.std(dim=['x','y'])

In [263]:
covars.shape

(13, 886, 659)

In [264]:
#save the standardised covariate raster - this will come in handy later on
with open("standardised_NN_covars.pkl","wb") as f:
    pickle.dump(covars,f)

## Generating labelled datasets for training and validation
We can now iterate through the dataframe and save the input data associated with each sample site in a directory in the 'normal' way for use with a Keras generator. This avoids loading every sample into RAM to train the NN. The labels are the measured SOC values in the dataframe. We will need to associate each row of the dataframe with a unique file on disk which can be read by the generator which feeds samples to Keras.

In [ ]:
from tensorflow.keras.utils import Sequence

In [73]:
len(final_df)

2183

In [80]:
final_df.iloc[0:10]['TC']

0    0.981252
1    0.600364
2    0.866419
3    1.187051
4    0.772519
5    1.398617
6    0.593211
7    1.126836
8    1.315066
9    1.881963
Name: TC, dtype: float64

In [312]:
class CovarGenerator(Sequence):
    """
    Feed trimmed covariate images for an NN
    """
    
    def __init__(self,gen_df,batch_size = 32, shuffle = True):
        self.batch_size = batch_size
        
        self.length = len(gen_df)//batch_size
        
        self.shuffle = shuffle
        
        if self.shuffle:
            self.gen_df = gen_df.sample(frac=1).reset_index(drop=True)
        else:
            self.gen_df = gen_df
        
    def __getitem__(self,index):
        slicedf = self.gen_df.iloc[index*self.batch_size:(index+1)*self.batch_size]
        y = np.array(slicedf['TC'])
        X = np.stack(slicedf.apply(sample_raster,axis=1))
        #channels-last is default for TF+Keras
        X = np.moveaxis(X,1,3)
        return (X,y)
        
    def __len__(self):
        return self.length
    
    def on_epoch_end(self):
        if self.shuffle:
            self.gen_df = self.gen_df.sample(frac=1).reset_index(drop=True)


In [293]:
testgen = CovarGenerator(final_df)

In [294]:
X,y = testgen[0]

In [295]:
X.shape

(32, 21, 21, 13)

In [274]:
len(testgen)

68

In [111]:
y.shape

(32,)

In [278]:
final_df = final_df.sample(frac=1).reset_index(drop=True)

In [279]:
train_df = final_df.iloc[0:int(0.7*len(final_df))]

In [280]:
val_df = final_df.iloc[int(0.7*len(final_df)):int(0.85*len(final_df))]

In [281]:
test_df = final_df.iloc[int(0.85*len(final_df)):]

In [313]:
training_gen = CovarGenerator(train_df)

In [314]:
validation_gen = CovarGenerator(val_df)

In [330]:
#CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.optimizers import Adam

In [331]:
model = Sequential()

model.add(Conv2D(8,kernel_size=10,activation='relu',input_shape=(21,21,13)))
model.add(SpatialDropout2D(0.5))
model.add(Flatten())
model.add(Dense(1,activation='relu'))

model.compile(optimizer='adam',loss='mse')



In [332]:
model.fit_generator(training_gen,epochs=50,validation_data=validation_gen)

Epoch 1/50
47/47 [==============================] - 21s 449ms/step - loss: 5.1021 - val_loss: 2.2716
Epoch 2/50
47/47 [==============================] - 19s 411ms/step - loss: 3.9467 - val_loss: 2.3584
Epoch 3/50
47/47 [==============================] - 18s 389ms/step - loss: 3.5312 - val_loss: 1.8127
Epoch 4/50
47/47 [==============================] - 18s 390ms/step - loss: 3.1296 - val_loss: 2.5660
Epoch 5/50
47/47 [==============================] - 17s 369ms/step - loss: 3.3275 - val_loss: 2.0768
Epoch 6/50
47/47 [==============================] - 19s 402ms/step - loss: 2.9234 - val_loss: 2.3308
Epoch 7/50
47/47 [==============================] - 17s 357ms/step - loss: 3.0341 - val_loss: 2.1161
Epoch 8/50
47/47 [==============================] - 18s 381ms/step - loss: 2.7500 - val_loss: 2.2516
Epoch 9/50
47/47 [==============================] - 18s 386ms/step - loss: 2.8533 - val_loss: 2.2947
Epoch 10/50
47/47 [==============================] - 17s 371ms/step - loss: 3.1796 - val_lo

In [333]:
testgen = CovarGenerator(test_df)
model.evaluate_generator(testgen)

1.9286025583744049

In [ ]:
model.predict_generator(testgen)

In [288]:
tsoc = 0
for batch,soc in training_gen:
    tsoc += np.mean(soc)
msoc = tsoc/len(training_gen)

In [289]:
msoc

2.5734063676475403

In [224]:
model.weights[0]

<tf.Variable 'conv2d_44/kernel:0' shape=(5, 5, 13, 8) dtype=float32>

In [227]:
covars.mean(dim=['x','y'])

<xarray.DataArray (channel: 13)>
array([ 1.619998e-16,  3.902013e-16,  7.919880e-16,  9.558375e-16,
       -6.707803e-17, -2.570189e-17,  2.267899e-16, -1.490125e-15,
        8.933352e-16,  5.919222e-17, -5.705040e-16,  6.489726e-16,
       -1.007047e-15])
Coordinates:
  * channel  (channel) int64 0 1 2 3 4 5 6 7 8 9 10 11 12

In [228]:
covars.std(dim=['x','y'])

<xarray.DataArray (channel: 13)>
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Coordinates:
  * channel  (channel) int64 0 1 2 3 4 5 6 7 8 9 10 11 12

In [268]:
covars.isel(x=300,y=300)

<xarray.DataArray (channel: 13)>
array([-0.037357, -0.243832,  0.477716,  0.557435,  0.268647,  0.097549,
       -0.154819, -0.255622, -0.324444,  0.154503,  0.104352,  0.116006,
       -0.331155])
Coordinates:
    y        float64 -3.704e+06
    x        float64 1.789e+06
  * channel  (channel) int64 0 1 2 3 4 5 6 7 8 9 10 11 12

In [2]:
import numpy as np
def sample_dc(row, dc, bufferx = 10, buffery = 10, xres = 25, yres = -25, product = 'fc_percentile_albers_annual', crs = 'EPSG:3577', **query):
    LL = row['points']
    sitex = LL.coords[0][0]
    sitey = LL.coords[0][1]
    
    x = np.arange(sitex-bufferx*xres,sitex+(bufferx+1)*xres,xres)
    y = np.arange(sitey-buffery*yres,sitey+(buffery+1)*yres,yres)
    
    covars = dc.load(product = product, x = (min((x[0],x[-1])),max((x[0],x[-1]))), y = (min((y[0],y[-1])),max((y[0],y[-1]))), crs = crs, **query)
    
    print(len(covars.variables))
    
    sample_array = covars.reindex(x=x,method='nearest',tolerance=abs(xres/2)).reindex(y=y,method='nearest',tolerance=abs(yres/2))
    
    sample_array = sample_array.fillna(0)
    
    return sample_array

In [5]:
import datacube
dc = datacube.Datacube()

In [4]:
trow = final_df.iloc[0]

In [5]:
test_xr = sample_dc(trow,dc).isel(time=0)

12


In [11]:
mband_np = test_xr.to_array().data

In [6]:
sitex,sitey = trow['points'].coords[0]

In [7]:
sitex

1784613.2479390604

In [8]:
sitey

-3710538.5739176488

In [11]:
query = {
    'x': (sitex-300,sitex+300),
    'y': (sitey-300,sitey+300),
    'time': ('2001-03-01','2002-03-01')
}

In [20]:
dc.load(product = 'fc_percentile_albers_annual',**query, crs='EPSG:3577')

<xarray.Dataset>
Dimensions:    (time: 1, x: 25, y: 25)
Coordinates:
  * time       (time) datetime64[ns] 2002-01-01
  * y          (y) float64 -3.71e+06 -3.71e+06 ... -3.711e+06 -3.711e+06
  * x          (x) float64 1.784e+06 1.784e+06 1.784e+06 ... 1.785e+06 1.785e+06
Data variables:
    BS_PC_10   (time, y, x) int8 0 0 4 18 19 22 19 7 4 4 ... 7 8 6 8 0 4 2 0 0 8
    PV_PC_10   (time, y, x) int8 47 33 19 10 8 8 13 33 ... 30 26 31 37 39 40 37
    NPV_PC_10  (time, y, x) int8 25 36 45 43 42 38 41 ... 21 14 18 17 21 19 17
    BS_PC_50   (time, y, x) int8 1 7 19 36 38 39 33 13 8 ... 9 13 11 8 7 6 8 18
    PV_PC_50   (time, y, x) int8 61 47 34 14 13 14 22 ... 38 41 40 44 50 52 47
    NPV_PC_50  (time, y, x) int8 37 44 48 49 47 46 44 ... 46 47 47 43 41 42 36
    BS_PC_90   (time, y, x) int8 6 17 31 45 44 48 42 23 ... 17 16 13 11 11 11 20
    PV_PC_90   (time, y, x) int8 70 57 51 29 27 21 32 ... 75 83 81 81 76 79 72
    NPV_PC_90  (time, y, x) int8 45 49 53 52 54 55 50 ... 53 57 56 54 50 50

In [16]:
prod = dc.list_products()

In [46]:
prod['crs']

id
36                                             EPSG:4326
88                                             EPSG:3577
32                                             EPSG:4326
99                                             EPSG:3577
53     GEOGCS["GEOCENTRIC DATUM of AUSTRALIA",DATUM["...
                             ...                        
97                                             EPSG:3577
100                                            EPSG:3577
85                                             EPSG:3577
101                                            EPSG:3577
84                                             EPSG:3577
Name: crs, Length: 77, dtype: object

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [73]:
trow['points']

Geometry(POINT (1784613.24793906 -3710538.57391765), EPSG:3577)

In [89]:
prod = dc.list_products()

In [92]:
prod[prod['name']=='fc_percentile_albers_annual']['spatial_dimensions'].array[0]

('y', 'x')

In [104]:
np.arange(-300.,25.,25.)

array([-300., -275., -250., -225., -200., -175., -150., -125., -100.,
        -75.,  -50.,  -25.,    0.])

In [88]:
trow['points'].to_crs(CRS('EPSG:4326'))

Geometry(POINT (151.270161902695 -32.7902091714388), EPSG:4326)

In [86]:
from datacube.utils.geometry import CRS

In [8]:
import tifffile

In [132]:
from PIL import Image

In [12]:
tifffile.imsave('testTIF.tif',data=mband_np)

In [16]:
final_df['points'][0]

Geometry(POINT (1784613.24793906 -3710538.57391765), EPSG:3577)

In [141]:
from datacube_NN import model_development
import imp
imp.reload(model_development)

<module 'datacube_NN.model_development' from '/g/data/r78/rlt118/datacube-NN/datacube_NN/model_development.py'>

In [142]:
harvester = model_development.DatacubeHarvester(dc,'fc_percentile_albers_annual',time=('2013-05-01','2014-05-01'))

In [143]:
harvester.resolutions

{'y': -25, 'x': 25}

In [144]:
harvester.pixel_buffer

(10, 10)

In [145]:
harvester.spatial_buffer

{'y': -250, 'x': 250}

In [146]:
sample = final_df.head()

In [171]:
harvester.process('test_data',sample,'points','TC')

In [169]:
sample = final_df.sample(n=10).reset_index(drop=True)

In [170]:
len(sample)

10

In [172]:
sample

,SampleID,Easting,Northing,TC,Method,Year,points
0,2008_tj2_0_10,340203.7000,6366900.300,4.219505,NIR,2008,"Geometry({'type': 'Point', 'coordinates': (178..."
1,2001_A85_0_10,339686.0490,6369372.820,2.478898,CNS,2001,"Geometry({'type': 'Point', 'coordinates': (178..."
2,2006_f148_0_10,338110.0000,6372270.000,1.215441,NIR,2006,"Geometry({'type': 'Point', 'coordinates': (178..."
3,d100MIR,341493.6842,6368399.017,2.776235,MIR,2004,"Geometry({'type': 'Point', 'coordinates': (178..."
4,HV299,343823.8940,6382086.010,3.454304,MIR,2004,"Geometry({'type': 'Point', 'coordinates': (179..."
5,B28MIR,340333.1410,6369596.270,2.250575,MIR,2002,"Geometry({'type': 'Point', 'coordinates': (178..."
6,hvd20_1,337484.8000,6382815.500,2.459440,CNS,2010,"Geometry({'type': 'Point', 'coordinates': (178..."
7,f073MIR,338499.0000,6372880.000,0.869703,MIR,2006,"Geometry({'type': 'Point', 'coordinates': (178..."
8,C102MIR,338868.4120,6369044.610,4.459809,MIR,2003,"Geometry({'type': 'Point', 'coordinates': (178..."
9,HV239,345163.5750,6369984.240,1.729756,MIR,2004,"Geometry({'type': 'Point', 'coordinates': (179..."


In [173]:
import tifffile
test_read_array = tifffile.imread('test_data/0.tif')

In [174]:
test_read_array.shape

(9, 21, 21)

In [175]:
tpt = sample.iloc[0]['points']
sitex,sitey = tpt.coords[0]

In [176]:
prod = dc.list_products()
prod[prod['name']=='fc_percentile_albers_annual'].crs

id
99    EPSG:3577
Name: crs, dtype: object

In [177]:
test_dc_xr = dc.load(product='fc_percentile_albers_annual',x=(sitex-267.5,sitex+267.5),y=(sitey-267.5,sitey+267.5),crs='EPSG:3577',time=('2013-05-01','2014-05-01')).to_array()

In [178]:
test_dc_xr = test_dc_xr.isel(time=0)

In [179]:
test_dc_xr = test_dc_xr.reindex(x=sitex+np.arange(-250,275,25),y=sitey+np.arange(250,-275,-25),method='nearest',tolerance=12.5)

In [180]:
test_dc_array = test_dc_xr.data

In [181]:
import numpy as np 
np.array_equal(test_dc_array,test_read_array)

True

In [183]:
test_read_array

array([[[ 7,  8,  9, ...,  2,  1,  2],
        [ 4,  1,  2, ...,  3,  3,  1],
        [ 1,  3,  8, ...,  2,  3,  1],
        ...,
        [ 0, 15,  4, ..., 11, 10,  7],
        [11,  8,  5, ..., 11,  6,  0],
        [ 8,  7,  1, ...,  5,  2,  8]],

       [[45, 42, 42, ..., 56, 56, 57],
        [47, 43, 53, ..., 55, 57, 58],
        [45, 46, 50, ..., 54, 52, 55],
        ...,
        [21, 40, 53, ..., 47, 48, 49],
        [36, 40, 51, ..., 43, 46, 47],
        [38, 44, 45, ..., 53, 47, 52]],

       [[25, 23, 14, ..., 27, 26, 28],
        [27, 23, 18, ..., 20, 19, 26],
        [27, 21, 21, ..., 32, 20, 23],
        ...,
        [40, 24, 26, ...,  4, 11, 15],
        [23, 39, 17, ...,  9,  8, 17],
        [24, 14, 27, ..., 15, 13,  9]],

       ...,

       [[17, 20, 23, ...,  6,  7,  7],
        [19, 23, 21, ..., 11,  9,  9],
        [24, 21, 16, ...,  9, 15,  7],
        ...,
        [17, 21, 12, ..., 22, 16, 12],
        [15, 13, 12, ..., 18, 18, 19],
        [15, 12, 10, ..., 13, 16

In [168]:
test_read_array

array([[[ 0,  4, 10, ..., 10,  7,  8],
        [ 2,  4,  6, ..., 13,  9, 11],
        [ 3,  4,  0, ..., 12,  3,  3],
        ...,
        [ 2,  4,  4, ..., 10, 11, 14],
        [ 3,  4,  7, ..., 14, 15, 16],
        [ 1,  3,  4, ..., 11, 12, 11]],

       [[57, 44, 41, ..., 41, 44, 45],
        [52, 46, 49, ..., 45, 43, 46],
        [55, 40, 35, ..., 44, 38, 26],
        ...,
        [58, 53, 50, ..., 39, 40, 36],
        [58, 56, 50, ..., 47, 43, 41],
        [61, 57, 51, ..., 53, 51, 52]],

       [[30, 30, 30, ..., 12, 16, 17],
        [29, 24, 23, ..., 12, 16, 10],
        [27, 32, 34, ..., 14, 17, 30],
        ...,
        [33, 28, 20, ..., 27, 22, 20],
        [30, 25, 26, ..., 19, 17,  9],
        [29, 28, 26, ..., 19, 17, 19]],

       ...,

       [[ 4, 14, 20, ..., 26, 22, 22],
        [ 6, 11, 12, ..., 23, 21, 23],
        [10, 10, 24, ..., 26, 21, 22],
        ...,
        [ 4,  5, 16, ..., 23, 23, 28],
        [ 4,  7, 22, ..., 19, 21, 25],
        [ 4,  6, 17, ..., 17, 19

In [162]:
test_dc_array

array([[[11, 21, 17, ...,  3,  4,  5],
        [ 5, 10, 14, ...,  3,  4,  4],
        [ 9,  9,  4, ...,  4,  4,  4],
        ...,
        [ 9,  6, 11, ..., 13,  9,  8],
        [10, 11,  9, ...,  9, 10,  8],
        [13,  8,  7, ..., 10,  9,  8]],

       [[43, 39, 35, ..., 37, 40, 36],
        [45, 41, 40, ..., 37, 37, 33],
        [44, 44, 44, ..., 35, 34, 30],
        ...,
        [12, 19, 32, ..., 51, 45, 32],
        [14, 23, 33, ..., 59, 48, 28],
        [30, 32, 34, ..., 55, 42, 28]],

       [[20, 19, 19, ..., 34, 18, 18],
        [28, 26, 26, ..., 34, 19, 18],
        [21, 23, 23, ..., 36, 19, 31],
        ...,
        [33, 30, 26, ..., 10, 13, 20],
        [31, 32, 27, ..., 10, 10, 20],
        [30, 29, 30, ..., 13, 16, 18]],

       ...,

       [[28, 34, 39, ..., 12, 12, 15],
        [22, 25, 29, ..., 12, 12, 14],
        [28, 20, 18, ..., 13, 13, 14],
        ...,
        [36, 33, 22, ..., 29, 25, 25],
        [41, 35, 25, ..., 28, 27, 25],
        [25, 23, 19, ..., 27, 27